In [ ]:
"""
Update Parameters Here
"""
CONTRACT_ADDRESS = "0x65b28ed75c12d8ce29d892de9f8304a6d2e176a7"

SAVE_RAW_DATA = False  # Set to True to keep each token raw json
COMPRESS_RAW_DATA = False  # Set to True to zip all raw json to save disk space

In [1]:
"""
@date: 11/01/2022
"""

# Collection name will be collected from raritysniffer response

import os
import requests
import time
import datetime
import json
import shutil
import pandas as pd

from honestnft_utils import config

dt_now = datetime.datetime.utcnow()
start_time = time.time()

print(f"Working in directory: {os.getcwd()}")

url = "https://raritysniffer.dev/api/v1/collection"

params = {
    "collection": CONTRACT_ADDRESS,
    "norm": True,
    "partial": False,
    "traitCount": True,
}

headers = {
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36",
}

response = requests.request("GET", url, headers=headers, params=params)
response_data = response.json()

# list to make completeness checks
token_ids = []

if response.status_code == 200:
    raw_metadata = []  # Initiate list of dicts that will be converted to DataFrame
    COLLECTION_NAME = response_data["name"].replace(" ", "")
    print(f"Received data for {COLLECTION_NAME}")
    print(f"{len(response_data['data'])} tokens in the collection")

    # Create folder to store metadata
    if SAVE_RAW_DATA:
        folder = f"{config.ATTRIBUTES_FOLDER}/{COLLECTION_NAME}/"
        print(f"Saving metadata to {folder}")
        if not os.path.exists(folder):
            os.mkdir(folder)

    for i, token in enumerate(response_data["data"]):
        token_ids.append(token["id"])
        # Add token name and token URI traits to the trait dictionary
        traits = dict()
        traits["TOKEN_ID"] = token["id"]
        traits["TOKEN_NAME"] = token["name"]

        # token['attributes'] = token.pop('traits')
        for atr in token["traits"]:
            if not atr["c"] == "Trait Count":
                traits[atr["c"]] = atr["n"]

        # add this token to dictionary list
        raw_metadata.append(traits)

        if SAVE_RAW_DATA:
            print(f"Saving raw attributes to disk...")
            PATH = f"{config.ATTRIBUTES_FOLDER}/{COLLECTION_NAME}/{token['id']}.json"
            with open(PATH, "w") as destination_file:
                json.dump(token, destination_file)
else:
    print(f"Got a {response.status_code} error from raritysniffer api")
    print("Exiting")

print(f"tokens in the list {len(token_ids)}")
print(f"lower_id: {min(token_ids)}")
print(f"upper_id: {max(token_ids)}")

if min(token_ids) == 0:
    max_supply = max(token_ids) + 1
    print(f"max supply: {max(token_ids) + 1}")
else:
    max_supply = max(token_ids)
    print(f"max supply: {max(token_ids)}")

# warn if raritysniffer doesn't have the full collection
# TODO: print a list of missing token ids
if len(token_ids) != max_supply:
    print(f"{max_supply - len(token_ids)} tokens with missing metadata")

# Generate traits DataFrame and save to disk as csv
trait_db = pd.DataFrame.from_records(raw_metadata)
trait_db = trait_db.set_index("TOKEN_ID")
trait_db = trait_db.sort_index()
print(trait_db.head())
trait_db.to_csv(f"{config.ATTRIBUTES_FOLDER}/{COLLECTION_NAME}.csv")

# Calculate rarity_data csv with rarity.py
print(
    f"Calculating rarity data with 'rarity.py' using the command: python3 rarity.py -collection {COLLECTION_NAME}"
)
os.system(f"python3 rarity.py -collection {COLLECTION_NAME}")

# Compress raw data and delete folder
if COMPRESS_RAW_DATA:
    print("Compressing raw metadata")
    dir_name = f"{config.ATTRIBUTES_FOLDER}/{COLLECTION_NAME}"
    shutil.make_archive(dir_name, "zip", dir_name)
    shutil.rmtree(dir_name)

print("--- %s seconds to process collection" % (round(time.time() - start_time, 1)))
print("finished.")

Working in directory: /home/flyingfish/honestnft-shenanigans/metadata
Received data for ChubbyKaijuDAO
9999 tokens in the collection
tokens in the list 9999
lower_id: 1
upper_id: 9999
max supply: 9999
                 TOKEN_NAME   1/1          Skin Background         Cloth  \
TOKEN_ID                                                                   
1         Chubby Kaiju #1-1  None     Music Red       Rare  Purple Lover   
2           Chubby Kaiju #2  None  Money Orange        SOS     Tentacles   
3         Chubby Kaiju #3-3  None  Money Orange       Rare         Clown   
4         Chubby Kaiju #4-4  None    Money Teal     Common             K   
5         Chubby Kaiju #5-5  None    Power Wolf       Rare     Seal Suit   

             Mouth             Eyes           Hats  
TOKEN_ID                                            
1         Gangster       Sunglasses  Safety Helmet  
2             Fish  Future Cyclopes     Cool Crown  
3          Rainbow           Pirate      Sleep Hat  
4